In [1]:
import numpy as np 
import pandas as pd 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [2]:
space_titanic_df = pd.read_csv("C:/Users/MSI/Desktop/CUAI/Basic/spaceship-titanic/NotEncoding_train.csv")
submission_df = pd.read_csv("C:/Users/MSI/Desktop/CUAI/Basic/spaceship-titanic/NotEncoding_Age_test.csv")
space_titanic_df.drop(columns= {"Age_group", "Unnamed: 0", "Cabin_number", "PassengerId"}, inplace= True)
submission_df.drop(columns= {"Unnamed: 0", "Name", "Cabin_number"}, inplace= True)

In [3]:
numerical_features_list = ["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]
scaler = StandardScaler()
for column in numerical_features_list:
    space_titanic_df[column] = scaler.fit_transform(space_titanic_df[[column]])

In [4]:
onehot_df = pd.get_dummies(space_titanic_df, columns=["HomePlanet","CryoSleep","Destination","Cabin_deck","Cabin_side", "VIP"], drop_first=True)
onehot_df["Transported"].replace({True:1,False:0}, inplace=True)

In [5]:
X_train = onehot_df.drop(labels = {'Transported'}, axis=1)
y_train = (1*onehot_df["Transported"]).replace(0,-1)

In [6]:
mode_imputer = SimpleImputer(strategy='most_frequent')
submission_df.iloc[:,:] = mode_imputer.fit_transform(submission_df)

numerical_features_list = ["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]
scaler = StandardScaler()
for column in numerical_features_list:
    submission_df[column] = scaler.fit_transform(submission_df[[column]])


one_submission = pd.get_dummies(submission_df, columns=["HomePlanet","CryoSleep","Destination","Cabin_deck","Cabin_side", "VIP"], drop_first=True)

In [7]:
param_grid ={ 'C':[1e-0,1e1,1e2], 
 'kernel':['rbf', 'sigmoid'], 
 'gamma':['scale'],
 'degree':[1,2,3],
 }

In [8]:
svc = SVC() 
clf = GridSearchCV(svc, param_grid,n_jobs=-1,verbose=2) 
clf.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'C': [1.0, 10.0, 100.0], 'degree': [1, 2, 3],
                         'gamma': ['scale'], 'kernel': ['rbf', 'sigmoid'],
                         'max_iter': [-1], 'tol': [1e-05]},
             verbose=2)

In [13]:
clf.best_params_

{'C': 1.0,
 'degree': 1,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'tol': 1e-05}

In [14]:
X_train

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Cabin_deck_B,Cabin_deck_C,Cabin_deck_D,Cabin_deck_E,Cabin_deck_F,Cabin_deck_G,Cabin_deck_T,Cabin_side_S,VIP_True
0,0.709437,-0.336053,-0.282036,-0.286182,-0.272278,-0.264380,1,0,0,0,1,1,0,0,0,0,0,0,0,0
1,-0.336717,-0.170980,-0.276396,-0.244361,0.215496,-0.225582,0,0,0,0,1,0,0,0,0,1,0,0,1,0
2,2.034566,-0.270932,1.958934,-0.286182,5.693843,-0.221173,1,0,0,0,1,0,0,0,0,0,0,0,1,1
3,0.290975,-0.336053,0.521981,0.334432,2.685461,-0.094199,1,0,0,0,1,0,0,0,0,0,0,0,1,0
4,-0.894666,0.122818,-0.238169,-0.033587,0.229711,-0.262616,0,0,0,0,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.848924,-0.336053,3.991224,-0.286182,1.187489,-0.199129,1,0,0,0,0,0,0,0,0,0,0,0,0,1
8689,-0.755179,-0.336053,-0.282036,-0.286182,-0.272278,-0.264380,0,0,1,1,0,0,0,0,0,0,1,0,1,0
8690,-0.197230,-0.336053,-0.282036,2.845325,-0.271389,-0.264380,0,0,0,0,1,0,0,0,0,0,1,0,1,0
8691,0.221232,-0.336053,0.375340,-0.286182,0.041354,2.588123,1,0,0,0,0,0,0,0,1,0,0,0,1,0


In [15]:
one_submission

,PassengerId,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,...,Destination_TRAPPIST-1e,Cabin_deck_B,Cabin_deck_C,Cabin_deck_D,Cabin_deck_E,Cabin_deck_F,Cabin_deck_G,Cabin_deck_T,Cabin_side_S,VIP_True
0,0013_01,-1.182216e-01,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,0,0,1,...,1,0,0,0,0,0,1,0,1,0
1,0018_01,-6.886014e-01,-0.357339,-0.277879,-0.312173,2.287504,-0.246712,0,0,0,...,1,0,0,0,0,1,0,0,1,0
2,0019_01,1.669682e-01,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,1,0,1,...,0,0,1,0,0,0,0,0,1,0
3,0021_01,6.660505e-01,-0.357339,4.121518,-0.312173,-0.104002,0.226648,1,0,0,...,1,0,1,0,0,0,0,0,1,0
4,0023_01,-6.173039e-01,-0.340723,-0.283840,0.832122,-0.267841,-0.246712,0,0,0,...,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,3.808606e-01,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,0,0,1,...,1,0,0,0,0,0,1,0,1,0
4273,9269_01,9.512404e-01,-0.357339,0.277095,-0.281538,-0.258790,-0.130193,0,0,0,...,1,0,0,0,0,1,0,0,1,0
4274,9271_01,-2.532995e-16,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,0,1,1,...,0,0,0,1,0,0,0,0,0,0
4275,9273_01,-2.532995e-16,-0.357339,1.491019,-0.312173,-0.267841,0.176479,1,0,0,...,1,0,0,1,0,0,0,0,0,0


In [16]:
passenger_list = one_submission['PassengerId'].copy()
prediction = clf.predict(one_submission.drop('PassengerId',axis=1))

submission_csv = pd.DataFrame({"PassengerId":passenger_list,"Transported":prediction})
submission_csv.loc[submission_csv['Transported']==1,'Transported'] = True 
submission_csv.loc[submission_csv['Transported']==-1,'Transported'] = False 
submission_csv.to_csv("C:/Users/MSI/Desktop/CUAI/Basic/spaceship-titanic/submission_0.80313.csv", index=False)